In [9]:
dialogues=['hey i am here ']

In [32]:
from lightrag.core.embedder import Embedder
from lightrag.core.types import ModelClientType
from lightrag.components.data_process.text_splitter import TextSplitter
from lightrag.components.data_process import  ToEmbeddings
from lightrag.core.container import Sequential
from lightrag.components.model_client import TransformersClient

model_kwargs = {
    "model": "thenlper/gte-base",
    "dimensions": 256,
    "encoding_format": "float",
}


text_splitter =TextSplitter (
    split_by= "word",
    chunk_size=50,
    chunk_overlap=10

)

local_embedder = Embedder(model_client=TransformersClient(), model_kwargs=model_kwargs)
embedder_transformer = ToEmbeddings(local_embedder, batch_size=1)
data_transformer = Sequential(text_splitter, embedder_transformer)
print(data_transformer)

Sequential(
  (0): TextSplitter(split_by=word, chunk_size=50, chunk_overlap=10)
  (1): ToEmbeddings(
    batch_size=1
    (embedder): Embedder(
      model_kwargs={'model': 'thenlper/gte-base', 'dimensions': 256, 'encoding_format': 'float'}, 
      (model_client): TransformersClient()
    )
    (batch_embedder): BatchEmbedder(
      (embedder): Embedder(
        model_kwargs={'model': 'thenlper/gte-base', 'dimensions': 256, 'encoding_format': 'float'}, 
        (model_client): TransformersClient()
      )
    )
  )
)


In [38]:
output = local_embedder("hello")
print(output.length, output.embedding_dim, output.is_normalized)
# 1 768 True

1 768 True


In [36]:
from lightrag.core.types import Document
def map_text_to_document(text1) -> Document:
    # it can be important to keep the original data's id
    return Document( text=text1)

In [37]:
text_as_documents = [map_text_to_document(text) for text in dialogues]
print(text_as_documents)

# apply data transformation to the documents
output = data_transformer(text_as_documents)
print(output)

[Document(id=d6ca1b00-dec6-49cd-9afe-44bcc392820c, text='hey i am here ', meta_data=None, vector=[], parent_doc_id=None, order=None, score=None)]


Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00, 11.16it/s]
Adding embeddings to documents from batch: 1it [00:00, ?it/s]

[Document(id=304ddd0a-656a-401d-88de-9fdba4d92b2f, text='hey i am here ', meta_data=None, vector='len: 768', parent_doc_id=d6ca1b00-dec6-49cd-9afe-44bcc392820c, order=0, score=None)]


In [15]:
from lightrag.core.db import LocalDB

save = LocalDB('document')
print(save)
dialogues=['hey i am here ']
save.load(dialogues)
print(save)
# dialog_turn_db.load(dialog_turns)
# print(dialog_turn_db)

LocalDB(name='document', items=[], transformed_items={}, transformer_setups={}, mapper_setups={})
LocalDB(name='document', items=['hey i am here '], transformed_items={}, transformer_setups={}, mapper_setups={})


In [25]:
key = "split_and_embed"
save.transform(data_transformer, map_fn=map_text_to_document, key=key)

print(save.transformed_items[key])
print('\n\n\n')
print(save.transformer_setups[key])
print('\n\n\n')
print(save.mapper_setups[key])

Batch embedding documents: 100%|██████████| 1/1 [00:00<00:00, 12.33it/s]
Adding embeddings to documents from batch: 1it [00:00, ?it/s]

[Document(id=4adf5d2a-911a-4da9-bfad-97ce551ef069, text='hey i am here ', meta_data=None, vector='len: 768', parent_doc_id=78b87b58-6af9-4876-81fe-937685d31e71, order=0, score=None)]




Sequential(
  (0): TextSplitter(split_by=word, chunk_size=50, chunk_overlap=10)
  (1): ToEmbeddings(
    batch_size=1
    (embedder): Embedder(
      model_kwargs={'model': 'thenlper/gte-base', 'dimensions': 256, 'encoding_format': 'float'}, 
      (model_client): TransformersClient()
    )
    (batch_embedder): BatchEmbedder(
      (embedder): Embedder(
        model_kwargs={'model': 'thenlper/gte-base', 'dimensions': 256, 'encoding_format': 'float'}, 
        (model_client): TransformersClient()
      )
    )
  )
)




<function map_text_to_document at 0x000002748F93EB60>


In [17]:
save.save_state(filepath='.storage/save.pkl')  # store pickle file against each tenant id 
reloaded_save = LocalDB.load_state(filepath='.storage/save.pkl')
print(str(save.__dict__) == str(reloaded_save.__dict__))

True


for each item in our saved db , we extract its vector 

In [40]:
from lightrag.components.retriever.faiss_retriever import FAISSRetriever

retriever = FAISSRetriever(top_k=1, embedder=local_embedder)
embeddings = [item.vector for item in save.transformed_items[key]]

for item in save.transformed_items[key]:
    print("items vector is ",item.vector)
retriever.build_index_from_documents(documents=embeddings)

input_str="Where are you"
# get the relevant documents
top_k_documents = retriever(input=input_str)

# get the relevant dialog turns
parent_doc_ids = set(
    [
        save.transformed_items[key][doc_index].parent_doc_id
        for doc_index in top_k_documents[0].doc_indices
    ]
)

# condition_fn = lambda item: item.id in parent_doc_ids
# fetched_dialog_turns = [item for item in save.items if condition_fn(item)]

items vector is  [0.026797287166118622, -0.01136690191924572, 0.0314859040081501, -0.003428468480706215, 0.042790889739990234, 0.0022864476777613163, 0.024491289630532265, 0.05440252646803856, -0.03300881013274193, -0.06548209488391876, -0.01684613525867462, -0.006660363171249628, -0.028457539156079292, -0.013646919280290604, -0.008673996664583683, 0.06507445871829987, 0.07111721485853195, -0.02005978859961033, 0.017987407743930817, 0.0010693282820284367, 0.0003830081259366125, -0.008108886890113354, 0.015240663662552834, 0.016948241740465164, 0.016532793641090393, -0.007366434205323458, 0.001936542335897684, -0.0007527291309088469, -0.08709736168384552, -0.025903629139065742, 0.027047794312238693, 0.004258498549461365, 0.011798429302871227, -0.04059164971113205, -0.0004150616587139666, -0.007818371057510376, -0.015595109201967716, -0.011842437088489532, -0.0064844852313399315, -0.025089144706726074, -0.041776757687330246, -0.02456466481089592, -0.02307419665157795, 0.01185645256191492

In [30]:
print("The retreiver is ",retriever)

The retreiver is  FAISSRetriever(
  top_k=1, metric=prob, dimensions=768, total_documents=1
  (embedder): Embedder(
    model_kwargs={'model': 'thenlper/gte-base', 'dimensions': 256, 'encoding_format': 'float'}, 
    (model_client): TransformersClient()
  )
)


In [29]:
print("top k documents are",top_k_documents)

top k documents are [RetrieverOutput(doc_indices=[0], doc_scores=[0.9300000071525574], query='Where are you', documents=None)]


In [1]:
import torch



